In [1]:
import pickle
import json
import numpy as np
import pickle as pkl
import os
from src.utils.data_transform import *
from src.utils.data_io import *
import importlib
from src.utils.cnn_steps import *
from src.utils.preprocessing import *
from src.utils.lstm_steps import *
from src.utils.stats import *

**Environment data setup**

In [2]:
with open("../data/dataset-info-json/subject_to_indices.json", "r") as f:
    subject_to_indices = json.load(f)
subject_to_indices = {int(k): v for k, v in subject_to_indices.items()}

predictions_path = "../data/cnn_predictions/full/timestamped/"
predictions = load_cnn_predictions(subject_to_indices, predictions_path)

with open("../data/original/FIC.pkl", "rb") as f:
    fic = pickle.load(f)
mm_gt = fic["mm_gt"]
sig = fic["signals_proc"]
bite_gt = fic["bite_gt"]

**Extract information on bites**

In [3]:
bite_lengths = []
for meal in bite_gt:
    for bite in meal:
        bite_lengths.append(bite[1] - bite[0])

In [4]:
bite_lengths = np.array(bite_lengths)

In [5]:
print(f'Average bite length: {np.mean(bite_lengths)}')
print(f'Min bite length: {np.min(bite_lengths)}')
print(f'Max bite length: {np.max(bite_lengths)}')

Average bite length: 4.521831081081081
Min bite length: 1.2100000000000364
Max bite length: 41.19


In [7]:
print(calculate_bite_stats(bite_gt))

{'Original Mean': 4.521831081081081, 'Original Std Dev': 3.226590120242883, 'Lower Bound': -1.023499999999986, 'Upper Bound': 8.844499999999995, 'Filtered Mean': 3.807725122349102, 'Filtered Std Dev': 1.595918424274311, 'Filtered Median': 3.390999999999991, 'Filtered Min': 1.2100000000000364, 'Filtered Max': 8.742999999999995}


**Create positive example bites**

In [42]:
# OG
positive_examples = {}
removed_bites = 0
for session_id in range(len(bite_gt)):
    print(f"Session: {session_id}\n")
    positive_examples[session_id], counter = create_positive_example_bites(predictions[session_id+1], bite_gt[session_id])
    removed_bites += counter
    if counter > 0:
        print(f"Removed {removed_bites} bites.")

Session: 0

Error in making bite windows. Expected length: 90, actual length: 132.
Info on window:		Start time: 1817) 188.856	End time: 1949) 202.052.


Error in making bite windows. Expected length: 90, actual length: 91.
Info on window:		Start time: 2321) 239.20600000000002	End time: 2412) 248.299.


Error in making bite windows. Expected length: 90, actual length: 102.
Info on window:		Start time: 2603) 267.422	End time: 2705) 277.677.


Error in making bite windows. Expected length: 90, actual length: 412.
Info on window:		Start time: 2932) 300.384	End time: 3344) 341.574.


Error in making bite windows. Expected length: 90, actual length: 125.
Info on window:		Start time: 4006) 407.774	End time: 4131) 420.204.


Error in making bite windows. Expected length: 90, actual length: 93.
Info on window:		Start time: 4131) 420.204	End time: 4224) 429.514.


Error in making bite windows. Expected length: 90, actual length: 91.
Info on window:		Start time: 4488) 455.922	End time: 4579) 465.

In [43]:
counter = 0
for i in range(21):
    counter = counter+len(positive_examples[i])
print(counter)

1233


**Create negative example bites**

In [3]:
negative_examples = {}
for session_id in range(len(bite_gt)):
    print(f"Session: {session_id}\n")
    negative_examples[session_id] = create_negative_example_bites(predictions[session_id+1], bite_gt[session_id])

Session: 0

Session: 1

Session: 2

Session: 3

Session: 4

Session: 5

Session: 6

Session: 7

Session: 8

Session: 9

Session: 10

Session: 11

Session: 12

Session: 13

Session: 14

Session: 15

Session: 16

Session: 17

Session: 18

Session: 19

Session: 20


In [40]:
counter = 0
for i in range(21):
    counter = counter+len(negative_examples[i])
print(counter)

41222


In [27]:
timestamps = [window[0][0, 5] for window in negative_examples[0]]

note: 114, 115 are all zeroes with label = 0. Why not just skip? investigate.

In [38]:
negative_examples[0][2715][0]

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0.

In [28]:
np.array(timestamps)

array([   7.195,    7.295,    7.395, ..., 1143.895, 1143.995, 1144.095])

In [7]:
bite_gt[0]

array([[  27.462,   29.88 ],
       [  29.88 ,   32.459],
       [  32.459,   35.019],
       [  36.069,   38.906],
       [  38.906,   41.659],
       [  41.659,   44.427],
       [  44.427,   47.626],
       [  47.626,   55.807],
       [  55.807,   58.776],
       [  58.776,   61.746],
       [  69.102,   72.475],
       [  83.216,   85.501],
       [  85.501,   88.355],
       [  91.875,   95.295],
       [  95.295,   99.014],
       [  99.014,  102.086],
       [ 106.307,  110.011],
       [ 151.219,  153.938],
       [ 153.938,  158.275],
       [ 158.275,  160.444],
       [ 161.729,  163.612],
       [ 163.612,  165.462],
       [ 165.462,  167.151],
       [ 179.58 ,  183.149],
       [ 188.856,  202.052],
       [ 231.499,  239.206],
       [ 239.206,  248.299],
       [ 267.422,  277.677],
       [ 300.384,  341.574],
       [ 360.412,  366.788],
       [ 366.788,  368.919],
       [ 383.402,  388.405],
       [ 389.452,  396.031],
       [ 397.263,  402.302],
       [ 407.7

**Testing**

In [ ]:
for i in range(0,21):
    print(f"True bite length: {len(bite_gt[i])}\tCalculated bite length: {len(positive_examples[i])}")

In [ ]:
true_bites_left_21 = [(bite[1]-bite[0]) for bite in bite_gt[20] if bite[1]- bite[0] <= 9]

In [ ]:
# Sessioned bite lengths
bites_per_session = {}
session_id = 0
for session in bite_gt:
    bite_lengths = []  # Initialize list to store bite lengths for the current meal
    for bite in session:
        bite_lengths.append(bite[1] - bite[0])
    bites_per_session[session_id] = bite_lengths  # Use meal ID as key
    session_id += 1  # Increment meal ID for the next meal

In [ ]:
true_bites_per_session_less_than_9 = {}
for session_id, bite_length in bites_per_session.items():
    counter = sum([1 for bite in bite_length if bite <= 9])
    true_bites_per_session_less_than_9[session_id] = counter

In [ ]:
calculated_bites_per_session_less_than_9 = {}
for session_id, windows in positive_examples.items():
    calculated_bites_per_session_less_than_9[session_id] = len(windows)

In [ ]:
bites_comparison = []

# Iterate through the session IDs in the true bites per session dictionary
for session_id, true_count in true_bites_per_session_less_than_9.items():
    calculated_count = calculated_bites_per_session_less_than_9.get(session_id, 0)
    # Append session ID, true count, and calculated count to the session data list
    bites_comparison.append([session_id, true_count, calculated_count])

# Convert the session data list into a NumPy array
bites_comparison = np.array(bites_comparison)

In [ ]:
positive_examples[17][23][0]

In [ ]:
np.vstack((bite_gt[17].T,[bite[1] - bite[0] for bite in bite_gt[17]])).T